In [ ]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community import embeddings

# from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [2]:
output_path = "./phesgo_refine"

In [3]:
raw_pdf_elements = partition_pdf(
    filename=r"C:\Users\Public\VS_Code_Workspace\Langchain_LLM\sriram\chatbot\chatbot\phesgorefined_scraped.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    extract_image_block_output_dir=output_path,
)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [5]:
text_elements = []
table_elements = []

text_summaries = []
table_summaries = []

summary_prompt = """
Summarize the following {element_type}:
{element}
"""
summary_chain = LLMChain(
    llm=ChatOllama(model="mistral",temperature=0 ),
    prompt=PromptTemplate.from_template(summary_prompt)
)
# after_rag_template = """
# # Summarize the following {element_type}:
# # {element}
# # """
# llm=ChatOllama(model="mistral",temperature=0)

# after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
# runnable = after_rag_template | llm | StrOutputParser()

for e in raw_pdf_elements:
    if 'CompositeElement' in repr(e):
        text_elements.append(e.text)
        summary = summary_chain.run({'element_type': 'text', 'element': e})
        text_summaries.append(summary)

    elif 'Table' in repr(e):
        table_elements.append(e.text)
        summary = summary_chain.run({'element_type': 'table', 'element': e})
        table_summaries.append(summary)

c:\Users\Barani\Desktop\chatbot\multimodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [6]:
output_path = "./phesgo_refined"

In [7]:
image_elements = []

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')
for i in os.listdir(output_path):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, i)
        encoded_image = encode_image(image_path)
        image_elements.append(encoded_image)


In [8]:
len(image_elements)

28

In [11]:
import os
import ollama
from tqdm import tqdm
import base64
from PIL import Image
from io import BytesIO

# Initialize an empty list to store the results
img_summaries = []


# Iterate over all image strings in the list with a tqdm progress bar
for image_string in tqdm(image_elements, desc="Processing images"):
    # Decode the image string using utf-8 encoders
    img_data = base64.b64decode(image_string)
    img = Image.open(BytesIO(img_data))
    
    # Save the image to a temporary file
    img.save("temp.jpg")
    
    # Call the chat function with the image file
    res = ollama.chat(
        model="llava",
        messages=[
            {
                'role':'user',
                'content':'Describe elaboratively about this image :',
                'images':["temp.jpg"]
            }
        ]
    )
    
    # Append the result to the image_summary list
    img_summaries.append(res['message']['content'])

# Print the image_summary list
print(img_summaries)


Processing images: 100%|██████████| 28/28 [16:01<00:00, 34.34s/it]

[' The image shows a screenshot of a website for the Health and Support Foundation. The website is designed with a clean, modern aesthetic. At the top, there\'s a navigation bar with links such as "Home," "About," "Support Us," and other sections typically found on a nonprofit organization\'s site. \n\nThe main section of the image features a woman prominently in the foreground. She appears to be middle-aged or older, with short hair, wearing a dark top that contrasts with her lighter skin tone. Her expression is serious and she seems focused on something not visible in the screenshot.\n\nBelow the woman\'s photo, there\'s a large banner with text. The banner includes a quote from someone named "Fraser" who states, "Having Hepatitis C and being able to access treatment has made a huge difference in my life." Below this quote is an image of a hand grasping the edge of a cliff. This graphic is likely intended to convey the idea that those with hepatitis C (HCV) are on a precipice, possib

In [ ]:
len(img_summaries)

28

In [ ]:
type(text_elements[0])

str

In [12]:
documents = []
retrieve_contents = []

for e, s in zip(text_elements, text_summaries):
    i = str(uuid.uuid4())
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'text',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)

for e, s in zip(table_elements, table_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'table',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)

for e, s in zip(image_elements, img_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'image',
            'original_content': e
        }
    )
    retrieve_contents.append((i, s))
    documents.append(doc)

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'))

In [13]:
vectorstore.save_local("phesgo_faiss2")

In [2]:
embeddings = embeddings.OllamaEmbeddings(model='nomic-embed-text')

In [3]:
db = FAISS.load_local("phesgo_faiss2", embeddings,allow_dangerous_deserialization=True)

In [4]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS

In [5]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [6]:
prompt_template = """
You are a data scientist and an expert in analyzing images and tables and text.
Answer the question based only on the following context, which can include text, images and tables:
{context}
Question: {question}
Don't answer if you are not sure and decline to answer and say "Sorry, I don't have much information about it."
Just return the helpful answer in as much as detailed possible.
Answer:
"""

In [7]:
model = ChatOllama(model="mistral",temperature=0)

In [8]:
# qa_chain = LLMChain(llm=ChatOllama(model="mistral", temperature=0),
#                         prompt=PromptTemplate.from_template(prompt_template))
after_rag_prompt = ChatPromptTemplate.from_template(prompt_template)
after_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        | model
        | StrOutputParser()
    )

In [9]:
async for chunk in after_rag_chain.astream("provide me the image that phesgo with perjeta and herceptin"):
    print(chunk, end="", flush=True)

 I cannot provide you with an image directly, but I can describe how these three medications - Phesgo (pertuzumab and trastuzumab delivered together via a single injection), Perjeta (pertuzumab), and Herceptin (trastuzumab) - are typically used in breast cancer treatment.

Phesgo is an FDA-approved combination of pertuzumab and trastuzumab administered as a single subcutaneous injection, while Perjeta and Herceptin are separate medications given intravenously. They are all used to treat HER2-positive breast cancer.

Perjeta (pertuzumab) is a monoclonal antibody that binds to the HER2 receptor on the surface of cancer cells, preventing them from growing and dividing. Herceptin (trastuzumab) is another monoclonal antibody that also targets the HER2 receptor on breast cancer cells, inhibiting their growth and division.

In combination therapy, Phesgo (pertuzumab and trastuzumab), Perjeta (pertuzumab), and Herceptin (trastuzumab) are used to improve treatment outcomes for patients with HER

In [10]:
# qa_chain = LLMChain(llm=ChatOllama(model="mistral", temperature=0),
#                         prompt=PromptTemplate.from_template(prompt_template))

def answer(question):
    relevant_docs = db.similarity_search(question)
    context = ""
    relevant_images = []
    for d in relevant_docs:
        if d.metadata['type'] == 'image':
            context += '[image]' + d.page_content
            relevant_images.append(d.metadata['original_content'])
    # result = qa_chain.run({'context': context, 'question': question})
    return relevant_images
question="provide me the image that phesgo with perjeta and herceptin"
async for chunk in after_rag_chain.astream(question):
    print(chunk, end="", flush=True)
rele_img=answer(question)
display.display(display.Image(base64.b64decode(rele_img[0])))

 I cannot provide you with an image directly, but I can describe how these three medications - Phesgo (pertuzumab and trastuzumab delivered together via a single injection), Perjeta (pertuzumab), and Herceptin (trastuzumab) - are typically used in breast cancer treatment.

Phesgo is an FDA-approved combination of pertuzumab and trastuzumab administered as a single subcutaneous injection, while Perjeta and Herceptin are separate medications given intravenously. They are all used to treat HER2-positive breast cancer.

Perjeta (pertuzumab) is a monoclonal antibody that binds to the HER2 receptor on the surface of cancer cells, preventing them from growing and dividing. Herceptin (trastuzumab) is another monoclonal antibody that also targets the HER2 receptor on breast cancer cells, inhibiting their growth and division.

In combination therapy, Phesgo (pertuzumab and trastuzumab), Perjeta (pertuzumab), and Herceptin (trastuzumab) are used to improve treatment outcomes for patients with HER

IndexError: list index out of range